In [1]:
# Preliminaries
import os
import numpy as np 
import pandas as pd

#transformers
from transformers import GPT2LMHeadModel
from transformers import GPT2Tokenizer

# Pytorch
import torch
import torch.nn as nn

#warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
# HElper Function
def choose_from_top(probs, n=5):
    ind = np.argpartition(probs, -n)[-n:]
    top_prob = probs[ind]
    top_prob = top_prob / np.sum(top_prob) # Normalize
    choice = np.random.choice(n, 1, p = top_prob)
    token_id = ind[choice][0]
    return int(token_id)

In [3]:
Tokenizer = GPT2Tokenizer.from_pretrained('gpt2-medium')

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [4]:
# Model Loading
model = GPT2LMHeadModel.from_pretrained('gpt2-medium')
special_tokens_dict = {'pad_token': '<PAD>','bos_token':'<soq>','sep_token':'<eoq>'}
num_added_toks = Tokenizer.add_special_tokens(special_tokens_dict)
print('We have added', num_added_toks, 'tokens')
model.resize_token_embeddings(len(Tokenizer))

Downloading:   0%|          | 0.00/718 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

We have added 3 tokens


Embedding(50260, 1024)

In [5]:
#loading Model state
models_path = "../input/training-humorous-sentence-completition/gpt2_medium_joker_3.pt" # ADD PATH TO YOUR SAVED MODEL HERE
model.load_state_dict(torch.load(models_path))

device='cuda'
model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50260, 1024)
    (wpe): Embedding(1024, 1024)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2):

In [6]:
def predict(start_of_joke,length_of_joke=96,number_of_jokes=2):
    joke_num = 0
    model.eval()
    with torch.no_grad():
        for joke_idx in range(number_of_jokes):
        
            joke_finished = False

            cur_ids = torch.tensor(Tokenizer.encode(start_of_joke)).unsqueeze(0).to(device)

            for i in range(length_of_joke):
                outputs = model(cur_ids, labels=cur_ids)
                loss, logits = outputs[:2]
                softmax_logits = torch.softmax(logits[0,-1], dim=0) #Take the first(from only one in this case) batch and the last predicted embedding
                if i < 3:
                    n = 20
                else:
                    n = 3
                next_token_id = choose_from_top(softmax_logits.to('cpu').numpy(), n=n) #Randomly(from the topN probability distribution) select the next word
                cur_ids = torch.cat([cur_ids, torch.ones((1,1)).long().to(device) * next_token_id], dim = 1) # Add the last word to the running sequence

                if next_token_id in Tokenizer.encode('<|endoftext|>'):
                    joke_finished = True
                    break

            
            if joke_finished:
                
                joke_num = joke_num + 1
                
                output_list = list(cur_ids.squeeze().to('cpu').numpy())
                output_text = Tokenizer.decode(output_list)

                print(output_text+'\n')

In [7]:
# Start Predicting
predict("How do you feel",64,1)

How do you feel when you hear a song that you <PAD>  can't sing <|endoftext|>



In [9]:
predict("Shit! I am alone",64,1)

Shit! I am alone with my wife. She has been drinking. She has a boyfriend. She is a slut. I have to get her out of my house. I have to get her out of the car. I don't have any money. I have no car! <|endoftext|>



In [14]:
predict("I am busy",64,1)

I am busy <PAD>  I just finished a job I was supposed to do. <|endoftext|>



In [11]:
predict("I will kill you",64,1)

I will kill you <eoq> I will make you a sandwich with my teeth <|endoftext|>



In [12]:
predict("My girlfriend broke up with me",64,1)

My girlfriend broke up with me <eoq> " and then she goes "I don't care, I'll never be able to tell you how much I love you" <|endoftext|>



In [13]:
predict("I would end up having no job",64,1)

I would end up having no job <eoq> I can't get a job. I'm not a lawyer. <|endoftext|>



In [22]:
predict("I would end up having no job",64,1)

I would end up having no job <soq>  Because I don't have a job. <|endoftext|>



In [18]:
predict("I wouldnt study",64,1)

I wouldnt study <PAD>  Because I would be too lazy. <|endoftext|>

